In [1]:
import cv2
import glob,os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [36]:
# reading images from the data folder
# label, smile = 1, neutral = 0
(img_width, img_height) = (45, 25)
def load_images_from_folder(folder):
    (images, lables, names, id) = ([], [], {}, 0)
    for (subdirs, dirs, files) in os.walk(folder):
        print subdirs, dirs
        for subdir in dirs:
            names[id] = subdir
            subjectpath = os.path.join(folder, subdir)
            for filename in os.listdir(subjectpath):
                path = subjectpath + '/' + filename
                lable = id
                img = cv2.imread(path, 0) # Reading each images in grayscale
                img = cv2.resize(img,(img_width, img_height)) # Resizing all the images
                images.append(img)
                lables.append(int(lable))
            id += 1
        
        return images, lables, names


In [37]:
X,Y,classes = load_images_from_folder("./data/")

(X,Y) = [np.array(lis) for lis in [X, Y]]
Y = pd.get_dummies(Y) #concerting labels to one-hot, Used Pandas for it. need to do it with numpy
# print Y
# print len(classes)


./data/ ['neutral', 'smile']


In [38]:
print "Height of the image:  " + str(X.shape[1])
print "Width of the image:   "  + str(X.shape[2])
print "Total number of data: "+ str(len(X))


Height of the image:  25
Width of the image:   45
Total number of data: 162


In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.40, random_state=42)

In [40]:
print "lenght of train data: "+str(len(X_train))
print "lenght of test data:  "+str(len(X_test))

lenght of train data: 97
lenght of test data:  65


In [41]:
train_set_x_flatten = X_train.reshape(X_train.shape[0],-1)
test_set_x_flatten = X_test.reshape(X_test.shape[0],-1)
print "train_set_x_flatten: "+str(train_set_x_flatten.shape)
print "test_set_x_flatten : "+str(test_set_x_flatten.shape)

print train_set_x_flatten

index = 35
# plt.imshow(X_train[index])
# print ("y = " + str(y_train[index]) + ", it's a '" + classes[np.squeeze(y_train[index])].decode("utf-8") +  "' picture.")

train_set_x_flatten: (97, 1125)
test_set_x_flatten : (65, 1125)
[[172 169 167 ..., 167 168 167]
 [137 120 122 ..., 129 132 128]
 [ 72  91  92 ...,  46  44  42]
 ..., 
 [150 150 143 ..., 160 158 159]
 [166 161 160 ..., 154 141 144]
 [ 27  29  30 ...,  88  84  60]]


In [42]:
#Normalizing data, conveting all pixel value in range between 0-1

train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.
print "length of train set: " + str(len(train_set_x))
print "length of test set: " + str(len(test_set_x))

length of train set: 97
length of test set: 65


In [43]:
print "shape of train set: "+ str(train_set_x.shape)
print "shape of test set: "+ str(test_set_x.shape)

print len(train_set_x), len(y_train)

shape of train set: (97, 1125)
shape of test set: (65, 1125)
97 97


# Building the softmax regression using tensorflow

In [44]:
import tensorflow as tf

In [90]:
x = tf.placeholder(tf.float32, [None, 1125])
W = tf.Variable(tf.zeros([1125, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y + 1e-50, labels = y_))


In [91]:
learning_rate = 0.1
print "training with learning rate: " + str(learning_rate)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for step in range(15000):
    trainStep,loss = sess.run([train_step, cross_entropy], feed_dict={x: train_set_x, y_: y_train})
    if step%1000==0:
        print "losses after per 1000 iteration: ",loss
        
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy with learning rate: " , sess.run(accuracy, feed_dict={x:test_set_x , y_: y_test}))

training with learning rate: 0.1
losses after per 1000 iteration:  0.693147
losses after per 1000 iteration:  0.0234265
losses after per 1000 iteration:  0.013271
losses after per 1000 iteration:  0.00945731
losses after per 1000 iteration:  0.00740099
losses after per 1000 iteration:  0.00610189
losses after per 1000 iteration:  0.00520222
losses after per 1000 iteration:  0.00454025
losses after per 1000 iteration:  0.00403168
losses after per 1000 iteration:  0.00362809
losses after per 1000 iteration:  0.00329963
losses after per 1000 iteration:  0.00302688
losses after per 1000 iteration:  0.0027966
losses after per 1000 iteration:  0.00259947
losses after per 1000 iteration:  0.00242874
('accuracy with learning rate: ', 0.93846154)


# prediction 

In [95]:
img = cv2.imread("/home/aquib/Desktop/smile_recognition/test/test_1.jpg", 0) # Reading each images in grayscale 
                                                                            # test_2 is a neutral face
                                                                            # test_1 is smile face
img = cv2.resize(img,(img_width, img_height)) # Resizing all the images
image=[]
image.append(img)
image_test = np.array(image)
test_image_flatten = image_test.reshape(image_test.shape[0],-1)
test_image_normalized = test_image_flatten/255.
print test_image_normalized.shape
#print len(test_image_normalized)

x_ = tf.cast(test_image_normalized, tf.float32)
y = tf.nn.softmax(tf.matmul(x_, W) + b)

result = sess.run(y)
indx = sess.run(tf.argmax(y))

print result
print indx

(1, 1125)
[[  7.46948572e-05   9.99925256e-01]]
[0 0]
